In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
data=pd.read_csv('/content/drive/MyDrive/HotelDatasetFinal.csv')

In [3]:
data.drop(columns=['Unnamed: 0','countyName','FacilitiesCount','NumFilteredFacilities','FilteredFacilitiesList'],inplace=True)

In [4]:
data.head(5)

,HotelName,Address,HotelFacilities,PhoneNumber,HotelWebsiteUrl,city,state,latitude,longitude,HotelRating
0,Sunbelt Lodge Motel,1903 Veterans Memorial Drive AbbevilleLouisian...,"['TV area meeting', 'Swimming pool', 'Smoke al...",3378981453,https://www.booking.com/hotel/us/sunbelt-lodge...,Abbeville,louisiana,-0.974812,0.012999,2.0
1,Budget Inn & Suites,2115 Charity Street AbbevilleLouisiana 70510,"['24-hour front desk ', 'Non-smoking rooms', '...",3378989770,https://www.booking.com/hotel/us/budget-inn-am...,Abbeville,louisiana,-0.976069,0.013095,2.0
2,Best Western Abbeville Inn & Suites,3515 Veterans Memorial Dr Abbeville AbbevilleL...,"['Wheelchair accessible', 'Vending machine', '...",3377404100,http://bestwesternlouisiana.com/hotels/best-we...,Abbeville,louisiana,-0.976959,0.014638,2.0
3,Motel 6 Abbeville LA,3523 Veterans Memorial Drive AbbevilleLouisian...,"['Wheelchair accessible', 'Wheelchair accessib...",3378983180,https://www.motel6.com/content/g6/motel6/motel...,Abbeville,louisiana,-0.976929,0.014690,2.0
4,Days Inn by Wyndham Abbeville,730 Veterans Memorial Dr AbbevilleLouisiana 70...,"['Express check', 'Multilingual staff', '24-ho...",3377404666,http://www.executiveinnexpress.com/,Abbeville,louisiana,-0.974707,0.012428,2.0


In [5]:
import re
def split_facilities(facilities):
    # Split based on capitalized words or known connectors like '/', ',', and words like 'for', 'and'
    # Use regex to split by commas or capitalized patterns that resemble phrases
    split_items = re.findall(r'(?<!\w)(?:\d+-hour [a-z\s]+|[A-Z][a-z]*(?:-[a-z]+)?(?:/[a-z]+)*(?:\s[a-z]+)*)(?!\w)', facilities)

    # Remove any duplicates or irrelevant short words
    unique_items = {item.strip() for item in split_items if len(item) > 2}
    return list(unique_items)

# Apply the function to the 'HotelFacilities' column
data['HotelFacilities'] = data['HotelFacilities'].apply(split_facilities)

# Display the first entry to see the result
print(data['HotelFacilities'][0])

['Business centre', 'On-site parking', 'Pets allowed internet services', 'Smoke alarms', 'Shallow end', 'Fence around pool', 'Heating', 'Fire extinguishers', 'Fax/photocopying outdoor pool', 'Non-smoking rooms', 'Daily housekeeping pool', 'Facilities for disabled guests', 'Free', '24-hour front desk', 'Air conditioning', 'Laundry', 'Swimming pool', 'Outdoor pool vending machine', 'Parking', 'Family rooms']


In [6]:
import re
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

In [7]:
spark = SparkSession.builder.appName('FacilityMapping').getOrCreate()

In [11]:
data_spark = spark.createDataFrame(data)

# Define the replacements dictionary
replacements = {
    'wi': 'Free Wifi',
    'wifi': 'Free Wifi',
    'pool': 'Pool',
    'outdoor pool':'Pool',
    'indoor pool': 'Pool',
    'front desk': '24 hr front desk',
    'registration desk': '24 hr front desk',
    'airport': 'Airport shuttle',
    'wheelchair': 'Wheel-chair accessible',
    'pets': 'Pet friendly',
    'kids': 'Child Friendly',
    'child':'Child Friendly',
    'bar': 'Open bar',
    'breakfast': 'Complementary breakfast',
    'check': 'Contactless check',
    'lounge': 'Private Lounge',
    'designated smoking': 'smoking allowed',
    'business': 'Business Center',
    'fitness': 'Fitness Center',
    'gym': 'Fitness Center',
    'disabled': 'Wheel-chair accessible',
    'supervised childcare': 'Child Friendly',
    'snacks': 'Snack Bar',
    'spa': 'Spa',
    'poolside': 'Open bar',
    'water-efficient': 'Eco-friendly',
    'energy-saving': 'Eco-friendly',
    'parking':'Valet Parking',
    'meeting rooms':'Business Center',
    'meeting room': 'Business Center',

}

# Mapping function
def map_facility(facilities):
    mapped_facilities = []
    for facility in facilities:
        facility_cleaned = facility.strip().lower()  # Clean up whitespace and convert to lowercase
        matched = False
        for keyword, replacement in replacements.items():
            if re.search(r'\b' + re.escape(keyword) + r'\b', facility_cleaned):  # Match the keyword in lowercase
                mapped_facilities.append(replacement)
                matched = True
                break
        if not matched:
            mapped_facilities.append('No smoking')  # Default if no keyword matches
    return mapped_facilities

# Create the pandas_udf to apply the function to each row
map_facility_udf = udf(map_facility, ArrayType(StringType()))

# Apply the UDF to the 'HotelFacilities' column in the PySpark DataFrame
data_spark = data_spark.withColumn('MappedFacilities', map_facility_udf('HotelFacilities'))

# Show the result with all original columns and the new 'MappedFacilities' column
data_spark.select('HotelName', 'Address', 'HotelFacilities', 'MappedFacilities', 'PhoneNumber', 'HotelWebsiteUrl', 'city', 'state', 'latitude', 'longitude', 'HotelRating').show(truncate=False)

+-----------------------------------------+-------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [12]:
row_count = data_spark.count()

# Print the result
print(f"Total number of rows: {row_count}")

Total number of rows: 103960


In [ ]:
# from pyspark.sql import functions as F

# # Count the number of rows where 'MappedFacilities' is either null or empty
# empty_mapped_facilities_count = data_spark.filter(
#     F.col('MappedFacilities').isNull() | (F.size(F.col('MappedFacilities')) == 0)
# ).count()

# # Print the result
# print(f"Number of rows with empty or null 'MappedFacilities': {empty_mapped_facilities_count}")


Number of rows with empty or null 'MappedFacilities': 31


In [ ]:
# from pyspark.sql import functions as F

# # Flatten the 'MappedFacilities' column into individual rows
# flattened_data = data_spark.select(F.explode('MappedFacilities').alias('Facility'))

# # Count the number of unique facilities
# unique_mapped_facilities_count = flattened_data.select('Facility').distinct().count()

# # Print the result
# print(f"Number of unique mapped facilities: {unique_mapped_facilities_count}")


In [ ]:
# from pyspark.sql import functions as F

# # Flatten the 'MappedFacilities' column into individual rows
# flattened_data = data_spark.select(F.explode('MappedFacilities').alias('Facility'))

# # Get the distinct unique facilities
# unique_mapped_facilities = flattened_data.select('Facility').distinct()

# # Show the unique mapped facilities
# unique_mapped_facilities.show(truncate=False)


In [13]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, StringType

# Define a UDF to remove duplicates within each row of the 'MappedFacilities' column
def remove_duplicates(facilities):
    return list(set(facilities)) if facilities else []

remove_duplicates_udf = F.udf(remove_duplicates, ArrayType(StringType()))

# Apply the UDF to the 'MappedFacilities' column
data_spark = data_spark.withColumn('MappedFacilities', remove_duplicates_udf(F.col('MappedFacilities')))

# Show the result to verify
data_spark.select('HotelName', 'MappedFacilities').show(truncate=False)


+-----------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|HotelName                                |MappedFacilities                                                                                                                                                                |
+-----------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Sunbelt Lodge Motel                      |[Pet friendly, Wheel-chair accessible, No smoking, Business Center, 24 hr front desk, Valet Parking, Pool]                                                                      |
|Budget Inn & Suites                      |[Free Wifi, No smoking, 24 hr front desk, Valet Parking, Pool]           

In [ ]:
# from pyspark.sql import functions as F

# # Filter rows where both 'No smoking' and 'Designated smoking room' appear in 'MappedFacilities'
# conflicting_rows = data_spark.filter(
#     F.array_contains(F.col('MappedFacilities'), 'No smoking') &
#     F.array_contains(F.col('MappedFacilities'), 'Designated smoking room')
# )

# # Show the rows with both facilities
# conflicting_rows.select('HotelName', 'MappedFacilities').show(truncate=False)


+---------+----------------+
|HotelName|MappedFacilities|
+---------+----------------+
+---------+----------------+



In [14]:
from pyspark.sql import functions as F

# Define default facilities to fill in if MappedFacilities is null or empty
default_facilities = ["Complementary breakfast", "24 hr front desk"]

# Fill null or empty values in 'MappedFacilities' with the default facilities
data_spark = data_spark.withColumn(
    'MappedFacilities',
    F.when(
        F.col('MappedFacilities').isNull() | (F.size(F.col('MappedFacilities')) == 0),
        F.array([F.lit(facility) for facility in default_facilities])
    ).otherwise(F.col('MappedFacilities'))
)

# Verify the result
data_spark.select('HotelName', 'MappedFacilities').show(truncate=False)


+-----------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|HotelName                                |MappedFacilities                                                                                                                                                                |
+-----------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Sunbelt Lodge Motel                      |[Pet friendly, Wheel-chair accessible, No smoking, Business Center, 24 hr front desk, Valet Parking, Pool]                                                                      |
|Budget Inn & Suites                      |[Free Wifi, No smoking, 24 hr front desk, Valet Parking, Pool]           

In [15]:
from pyspark.sql import functions as F

# Count the number of rows where 'MappedFacilities' is either null or empty
empty_mapped_facilities_count = data_spark.filter(
    F.col('MappedFacilities').isNull() | (F.size(F.col('MappedFacilities')) == 0)
).count()

# Print the result
print(f"Number of rows with empty or null 'MappedFacilities': {empty_mapped_facilities_count}")


Number of rows with empty or null 'MappedFacilities': 0


In [16]:
row_count = data_spark.count()

# Print the result
print(f"Total number of rows: {row_count}")

Total number of rows: 103960


In [17]:
data_spark.show(5)

+--------------------+--------------------+--------------------+-----------+--------------------+---------+---------+-------------------+------------------+-----------+--------------------+
|           HotelName|             Address|     HotelFacilities|PhoneNumber|     HotelWebsiteUrl|     city|    state|           latitude|         longitude|HotelRating|    MappedFacilities|
+--------------------+--------------------+--------------------+-----------+--------------------+---------+---------+-------------------+------------------+-----------+--------------------+
| Sunbelt Lodge Motel|1903 Veterans Mem...|[Business centre,...| 3378981453|https://www.booki...|Abbeville|louisiana|-0.9748122467166942|0.0129992214010715|        2.0|[Pet friendly, Wh...|
| Budget Inn & Suites|2115 Charity Stre...|[Non-smoking room...| 3378989770|https://www.booki...|Abbeville|louisiana|-0.9760694016374012|0.0130953274405426|        2.0|[Free Wifi, No sm...|
|Best Western Abbe...|3515 Veterans Mem...|[Assist

In [18]:
# Remove the 'HotelFacilities' column
data_spark = data_spark.drop('HotelFacilities')

# Show the result to verify the column has been removed
data_spark.show(truncate=False)


+-----------------------------------------+-------------------------------------------------------------------------------------+-----------+-----------------------------------------------------------------------------------------------------------------+----------+---------+-------------------+------------------+-----------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|HotelName                                |Address                                                                              |PhoneNumber|HotelWebsiteUrl                                                                                                  |city      |state    |latitude           |longitude         |HotelRating|MappedFacilities                                                                                                                                                        

## Getting rid of unnecessary rows


In [20]:
# Read the JSON file into a Spark DataFrame
df = spark.read.json("/content/city_state.json")

# Show the content of the DataFrame
df.show(truncate=False)

+---------------+----------------+-----+
|_corrupt_record|city            |state|
+---------------+----------------+-----+
|[              |NULL            |NULL |
|NULL           |Jefferson       |LA   |
|NULL           |Franklin        |MO   |
|NULL           |Lake            |IN   |
|NULL           |Westtown        |PA   |
|NULL           |Glendora        |NJ   |
|NULL           |Swedesboro      |NJ   |
|NULL           |Tom Green       |PA   |
|NULL           |St. Peters      |PA   |
|NULL           |Saginaw         |MO   |
|NULL           |Mount Laurel    |NJ   |
|NULL           |Blooming Glen   |PA   |
|NULL           |Fairview Heights|IL   |
|NULL           |Mansfield       |NJ   |
|NULL           |Edgewater Park  |NJ   |
|NULL           |Brooklawn       |NJ   |
|NULL           |Sewell          |NJ   |
|NULL           |Ewing           |NJ   |
|NULL           |West Point      |PA   |
|NULL           |Fayette         |TN   |
+---------------+----------------+-----+
only showing top

In [21]:
# Step 1: Remove the '_corrupt_record' column
df_cleaned = df.drop('_corrupt_record')

# Step 2: Filter out rows where 'city' or 'state' is NULL
df_cleaned = df_cleaned.filter(df_cleaned.city.isNotNull() & df_cleaned.state.isNotNull())

# Show the cleaned DataFrame
df_cleaned.show(truncate=False)


+----------------+-----+
|city            |state|
+----------------+-----+
|Jefferson       |LA   |
|Franklin        |MO   |
|Lake            |IN   |
|Westtown        |PA   |
|Glendora        |NJ   |
|Swedesboro      |NJ   |
|Tom Green       |PA   |
|St. Peters      |PA   |
|Saginaw         |MO   |
|Mount Laurel    |NJ   |
|Blooming Glen   |PA   |
|Fairview Heights|IL   |
|Mansfield       |NJ   |
|Edgewater Park  |NJ   |
|Brooklawn       |NJ   |
|Sewell          |NJ   |
|Ewing           |NJ   |
|West Point      |PA   |
|Fayette         |TN   |
|Philadelphia    |PA   |
+----------------+-----+
only showing top 20 rows



In [22]:
df_cleaned = df_cleaned.withColumn("city", F.lower(F.col("city")))

In [23]:
data_spark = data_spark.withColumn("city", F.lower(F.col("city")))


In [24]:
data_spark.show()


+--------------------+--------------------+-----------+--------------------+----------+---------+-------------------+------------------+-----------+--------------------+
|           HotelName|             Address|PhoneNumber|     HotelWebsiteUrl|      city|    state|           latitude|         longitude|HotelRating|    MappedFacilities|
+--------------------+--------------------+-----------+--------------------+----------+---------+-------------------+------------------+-----------+--------------------+
| Sunbelt Lodge Motel|1903 Veterans Mem...| 3378981453|https://www.booki...| abbeville|louisiana|-0.9748122467166942|0.0129992214010715|        2.0|[Pet friendly, Wh...|
| Budget Inn & Suites|2115 Charity Stre...| 3378989770|https://www.booki...| abbeville|louisiana|-0.9760694016374012|0.0130953274405426|        2.0|[Free Wifi, No sm...|
|Best Western Abbe...|3515 Veterans Mem...| 3377404100|http://bestwester...| abbeville|louisiana|-0.9769590285826324|0.0146380822846817|        2.0|[C

In [25]:
df_cleaned.show()

+----------------+-----+
|            city|state|
+----------------+-----+
|       jefferson|   LA|
|        franklin|   MO|
|            lake|   IN|
|        westtown|   PA|
|        glendora|   NJ|
|      swedesboro|   NJ|
|       tom green|   PA|
|      st. peters|   PA|
|         saginaw|   MO|
|    mount laurel|   NJ|
|   blooming glen|   PA|
|fairview heights|   IL|
|       mansfield|   NJ|
|  edgewater park|   NJ|
|       brooklawn|   NJ|
|          sewell|   NJ|
|           ewing|   NJ|
|      west point|   PA|
|         fayette|   TN|
|    philadelphia|   PA|
+----------------+-----+
only showing top 20 rows



In [26]:
# State name to code mapping (with lowercase keys)
state_mapping = {
    'alabama': 'AL', 'alaska': 'AK', 'arizona': 'AZ', 'arkansas': 'AR',
    'california': 'CA', 'colorado': 'CO', 'connecticut': 'CT', 'delaware': 'DE',
    'florida': 'FL', 'georgia': 'GA', 'hawaii': 'HI', 'idaho': 'ID',
    'illinois': 'IL', 'indiana': 'IN', 'iowa': 'IA', 'kansas': 'KS',
    'kentucky': 'KY', 'louisiana': 'LA', 'maine': 'ME', 'maryland': 'MD',
    'massachusetts': 'MA', 'michigan': 'MI', 'minnesota': 'MN', 'mississippi': 'MS',
    'missouri': 'MO', 'montana': 'MT', 'nebraska': 'NE', 'nevada': 'NV',
    'new hampshire': 'NH', 'new jersey': 'NJ', 'new mexico': 'NM', 'new york': 'NY',
    'north carolina': 'NC', 'north dakota': 'ND', 'ohio': 'OH', 'oklahoma': 'OK',
    'oregon': 'OR', 'pennsylvania': 'PA', 'rhode island': 'RI', 'south carolina': 'SC',
    'south dakota': 'SD', 'tennessee': 'TN', 'texas': 'TX', 'utah': 'UT',
    'vermont': 'VT', 'virginia': 'VA', 'washington': 'WA', 'west virginia': 'WV',
    'wisconsin': 'WI', 'wyoming': 'WY'
}

# Step 1: Create a UDF (User Defined Function) to map state names to codes
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def map_state_to_code(state_name):
    # Convert the state name to lowercase and get the code from the mapping
    return state_mapping.get(state_name.lower(), state_name.lower())  # Use lowercase matching

# Register the UDF with Spark
map_state_udf = udf(map_state_to_code, StringType())

# Step 2: Apply the UDF to transform the 'state' column (convert the state names to codes) directly in data_spark
data_spark = data_spark.withColumn('state', map_state_udf('state'))

# Show the cleaned DataFrame with state codes
data_spark.show(truncate=False)


+-----------------------------------------+-------------------------------------------------------------------------------------+-----------+-----------------------------------------------------------------------------------------------------------------+----------+-----+-------------------+------------------+-----------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|HotelName                                |Address                                                                              |PhoneNumber|HotelWebsiteUrl                                                                                                  |city      |state|latitude           |longitude         |HotelRating|MappedFacilities                                                                                                                                                                

In [27]:
row_count = data_spark.count()

# Print the result
print(f"Total number of rows: {row_count}")

Total number of rows: 103960


In [49]:
# Assuming `df_cleaned` also has a column 'city'
final_spark = data_spark.join(df_cleaned.select("city").distinct(), on="city", how="inner")


In [57]:
final_spark = final_spark.join(df_cleaned.select("state").distinct(), on="state", how="inner")

In [50]:
final_spark.show()

+-------+--------------------+--------------------+-----------+--------------------+-----+-------------------+-------------------+-----------+--------------------+
|   city|           HotelName|             Address|PhoneNumber|     HotelWebsiteUrl|state|           latitude|          longitude|HotelRating|    MappedFacilities|
+-------+--------------------+--------------------+-----------+--------------------+-----+-------------------+-------------------+-----------+--------------------+
|    ada|La Quinta Inn & S...|2828 East Arlingt...| 5804365000|http://www.lq.com...|   OK|-0.1646695541437174|-0.2411360582593694|        3.0|[Complementary br...|
|    ada|Hampton Inn & Sui...|1220 Lonnie Abbot...| 5804364040|http://hamptoninn...|   OK|-0.1631258691484655|-0.2425619121900004|        2.0|[Complementary br...|
|    ada|Holiday Inn Expre...|1201 Lonnie Abbot...| 5803109200|http://www.ihg.co...|   OK|-0.1631535434312835|-0.2425489856466796|        2.0|[Complementary br...|
|    ada|America

In [39]:
# print(type(final_spark['state']))

<class 'pyspark.sql.column.Column'>


In [59]:
# # Get unique states from final_spark
# unique_states_final_spark = final_spark.select('state').distinct()

# # Show the unique states
# unique_states_final_spark.show(truncate=False)


+-----+
|state|
+-----+
|AZ   |
|LA   |
|NJ   |
|ID   |
|CA   |
|NC   |
|MO   |
|IL   |
|IN   |
|TN   |
|PA   |
|CO   |
|FL   |
|NV   |
|DE   |
+-----+



In [45]:
# # Get unique states from final_spark (PySpark DataFrame)
# unique_states_final = set(final_spark.select('state').distinct().rdd.flatMap(lambda x: x).collect())

# # Get unique states from df_cleaned (also a PySpark DataFrame)
# unique_states_df_cleaned = set(df_cleaned.select('state').distinct().rdd.flatMap(lambda x: x).collect())

# # Find states in final_spark that are not in df_cleaned
# states_not_in_df_cleaned = unique_states_final - unique_states_df_cleaned

# # Display the results
# print("States in final_spark not in df_cleaned:", states_not_in_df_cleaned)


States in final_spark not in df_cleaned: {'KY', 'UT', 'WY', 'OH', 'NY', 'WV', 'KS', 'VA', 'SD', 'MN', 'WI', 'AL', 'NH', 'AR', 'GA', 'OR', 'MD', 'NM', 'IA', 'NE', 'MT', 'SC', 'VT', 'MS', 'CT', 'TX', 'MA', 'OK', 'MI', 'WA'}


In [60]:
row_count = final_spark.count()

# Print the result
print(f"Total number of rows: {row_count}")

Total number of rows: 3586


In [32]:
# unique_city_count = final_spark.select("city").distinct().count()
# print("Count of unique cities:", unique_city_count)


Count of unique cities: 114


In [30]:
# unique_city_count = df_cleaned.select("city").distinct().count()
# print("Count of unique cities:", unique_city_count)


Count of unique cities: 235


In [ ]:
# data_spark.filter(F.col("city") == "w. berlin|").show()


+----+---------+-------+-----------+---------------+-----+--------+---------+-----------+----------------+
|city|HotelName|Address|PhoneNumber|HotelWebsiteUrl|state|latitude|longitude|HotelRating|MappedFacilities|
+----+---------+-------+-----------+---------------+-----+--------+---------+-----------+----------------+
+----+---------+-------+-----------+---------------+-----+--------+---------+-----------+----------------+



In [ ]:
# final_spark = data_spark.join(df_cleaned, on="city", how="inner")

In [ ]:
# unique_city_count = final_spark.select("city").distinct().count()
# print("Count of unique cities:", unique_city_count)

Count of unique cities: 114


In [ ]:
# df_cleaned.filter(F.col("city") == "ada").show()


+----+-----+
|city|state|
+----+-----+
| ada|   ID|
+----+-----+



## Working on final df

In [61]:
final_spark.show()

+-----+----------+--------------------+--------------------+-----------+--------------------+------------------+-------------------+-----------+--------------------+
|state|      city|           HotelName|             Address|PhoneNumber|     HotelWebsiteUrl|          latitude|          longitude|HotelRating|    MappedFacilities|
+-----+----------+--------------------+--------------------+-----------+--------------------+------------------+-------------------+-----------+--------------------+
|   TN|   antioch|Hampton Inn Nashv...|210 Crossings Pla...| 6157319911|http://hamptoninn...|0.0484467229271417| 0.3202181942432139|        2.0|[Complementary br...|
|   TN|   antioch|Days Inn Nashvill...|510B Collins Park...| 6157319911|https://www.booki...|0.0484519540415768| 0.3198029711908092|        3.0|[Contactless chec...|
|   TN|   antioch|Comfort Inn & Suites|400 Collins Park ...| 6157318540|http://www.countr...|0.0488881614872188| 0.3195269051873696|        2.0|[Complementary br...|
|   

In [62]:
from pyspark.sql import functions as F

# Step 1: Get a distinct list of all facilities in the `MappedFacilities` column
# Explode the array and get unique facility names
unique_facilities = (final_spark
                     .select(F.explode("MappedFacilities").alias("facility"))
                     .distinct()
                     .rdd.flatMap(lambda x: x)
                     .collect())

# Step 2: For each unique facility, create a new column with 1 if present, else 0
for facility in unique_facilities:
    final_spark = final_spark.withColumn(facility,
                                       F.when(F.array_contains(F.col("MappedFacilities"), facility), 1).otherwise(0))

# Drop the original `MappedFacilities` column if needed
final_spark = final_spark.drop("MappedFacilities")

# Show the resulting DataFrame
final_spark.show()


+-----+----------+--------------------+--------------------+-----------+--------------------+------------------+-------------------+-----------+---+---------------+--------------+---------------+-----------------+----+----------+------------+---------+----------------------+---------+-----------------------+--------------+------------+----------------+---------------+-------------+--------+--------------+
|state|      city|           HotelName|             Address|PhoneNumber|     HotelWebsiteUrl|          latitude|          longitude|HotelRating|Spa|Airport shuttle|Private Lounge|Business Center|Contactless check|Pool|No smoking|Pet friendly|Free Wifi|Wheel-chair accessible|Snack Bar|Complementary breakfast|Fitness Center|Eco-friendly|24 hr front desk|smoking allowed|Valet Parking|Open bar|Child Friendly|
+-----+----------+--------------------+--------------------+-----------+--------------------+------------------+-------------------+-----------+---+---------------+--------------+---

In [63]:
# Filter rows where both 'SmokingAllowed' and 'NoSmoking' columns have a value of 1
count = final_spark.filter((F.col("smoking allowed") == 0) & (F.col("No smoking") == 0)).count()

# Print the result
print("Number of rows where both SmokingAllowed and NoSmoking are 1:", count)


Number of rows where both SmokingAllowed and NoSmoking are 1: 8


In [64]:
from pyspark.sql import functions as F

# Update 'SmokingAllowed' to 1 and 'NoSmoking' to 0 where both are 1
final_spark = final_spark.withColumn(
    "smoking allowed",
    F.when((F.col("smoking allowed") == 1) & (F.col("No smoking") == 1), 1).otherwise(F.col("smoking allowed"))
)

final_spark = final_spark.withColumn(
    "No smoking",
    F.when((F.col("smoking allowed") == 1) & (F.col("No smoking") == 1), 0).otherwise(F.col("No smoking"))
)

# Show the updated DataFrame
final_spark.show()


+-----+----------+--------------------+--------------------+-----------+--------------------+------------------+-------------------+-----------+---+---------------+--------------+---------------+-----------------+----+----------+------------+---------+----------------------+---------+-----------------------+--------------+------------+----------------+---------------+-------------+--------+--------------+
|state|      city|           HotelName|             Address|PhoneNumber|     HotelWebsiteUrl|          latitude|          longitude|HotelRating|Spa|Airport shuttle|Private Lounge|Business Center|Contactless check|Pool|No smoking|Pet friendly|Free Wifi|Wheel-chair accessible|Snack Bar|Complementary breakfast|Fitness Center|Eco-friendly|24 hr front desk|smoking allowed|Valet Parking|Open bar|Child Friendly|
+-----+----------+--------------------+--------------------+-----------+--------------------+------------------+-------------------+-----------+---+---------------+--------------+---

In [65]:
from pyspark.sql import functions as F

# Update 'SmokingAllowed' to 1 where both 'smoking allowed' and 'No smoking' are 0
final_spark = final_spark.withColumn(
    "smoking allowed",
    F.when((F.col("smoking allowed") == 0) & (F.col("No smoking") == 0), 1).otherwise(F.col("smoking allowed"))
)

# Show the updated DataFrame
final_spark.show()

+-----+----------+--------------------+--------------------+-----------+--------------------+------------------+-------------------+-----------+---+---------------+--------------+---------------+-----------------+----+----------+------------+---------+----------------------+---------+-----------------------+--------------+------------+----------------+---------------+-------------+--------+--------------+
|state|      city|           HotelName|             Address|PhoneNumber|     HotelWebsiteUrl|          latitude|          longitude|HotelRating|Spa|Airport shuttle|Private Lounge|Business Center|Contactless check|Pool|No smoking|Pet friendly|Free Wifi|Wheel-chair accessible|Snack Bar|Complementary breakfast|Fitness Center|Eco-friendly|24 hr front desk|smoking allowed|Valet Parking|Open bar|Child Friendly|
+-----+----------+--------------------+--------------------+-----------+--------------------+------------------+-------------------+-----------+---+---------------+--------------+---

In [66]:
same_values_count = final_spark.filter(F.col("No smoking") == F.col("smoking allowed")).count()

# Check if all rows have the same values in both columns
if same_values_count == data_spark.count():
    print("All rows have the same values in 'No smoking' and 'smoking allowed'.")
else:
    print("There are rows where 'No smoking' and 'smoking allowed' have different values.")
    print(f"Number of rows with the same values: {same_values_count}")

There are rows where 'No smoking' and 'smoking allowed' have different values.
Number of rows with the same values: 0


In [67]:
from pyspark.sql import functions as F

# Create a new column based on conditions
final_spark = final_spark.withColumn(
    "smoking",
    F.when(F.col("No smoking") == 1, 0)
    .when(F.col("smoking allowed") == 1, 1)
    .otherwise(F.lit(None))  # Set to None if neither condition is met
)

# Show the resulting DataFrame
final_spark.show()

+-----+----------+--------------------+--------------------+-----------+--------------------+------------------+-------------------+-----------+---+---------------+--------------+---------------+-----------------+----+----------+------------+---------+----------------------+---------+-----------------------+--------------+------------+----------------+---------------+-------------+--------+--------------+-------+
|state|      city|           HotelName|             Address|PhoneNumber|     HotelWebsiteUrl|          latitude|          longitude|HotelRating|Spa|Airport shuttle|Private Lounge|Business Center|Contactless check|Pool|No smoking|Pet friendly|Free Wifi|Wheel-chair accessible|Snack Bar|Complementary breakfast|Fitness Center|Eco-friendly|24 hr front desk|smoking allowed|Valet Parking|Open bar|Child Friendly|smoking|
+-----+----------+--------------------+--------------------+-----------+--------------------+------------------+-------------------+-----------+---+---------------+--

In [68]:
none_count = final_spark.filter(F.col("smoking").isNull()).count()

# Print the result
print("Number of rows where smoking_policy is None:", none_count)

Number of rows where smoking_policy is None: 0


In [69]:
# Drop the 'No smoking' and 'smoking allowed' columns
final_spark = final_spark.drop("No smoking", "smoking allowed")

# Show the resulting DataFrame
final_spark.show()


+-----+----------+--------------------+--------------------+-----------+--------------------+------------------+-------------------+-----------+---+---------------+--------------+---------------+-----------------+----+------------+---------+----------------------+---------+-----------------------+--------------+------------+----------------+-------------+--------+--------------+-------+
|state|      city|           HotelName|             Address|PhoneNumber|     HotelWebsiteUrl|          latitude|          longitude|HotelRating|Spa|Airport shuttle|Private Lounge|Business Center|Contactless check|Pool|Pet friendly|Free Wifi|Wheel-chair accessible|Snack Bar|Complementary breakfast|Fitness Center|Eco-friendly|24 hr front desk|Valet Parking|Open bar|Child Friendly|smoking|
+-----+----------+--------------------+--------------------+-----------+--------------------+------------------+-------------------+-----------+---+---------------+--------------+---------------+-----------------+----+--

In [71]:
facility_cols=['Spa', 'Airport shuttle','Private Lounge','Business Center','Contactless check', 'Pool','Pet friendly','Free Wifi','Wheel-chair accessible','Snack Bar','Complementary breakfast','Fitness Center','Eco-friendly','24 hr front desk','Valet Parking','Open bar','Child Friendly','smoking']

In [73]:
facility_counts = final_spark.select([F.sum(col).alias(col) for col in final_spark.columns])

# Show the result sorted by the most frequent facilities
facility_counts.show(truncate=False)

+-----+----+---------+-------+------------------+---------------+------------------+-----------------+-----------+---+---------------+--------------+---------------+-----------------+----+------------+---------+----------------------+---------+-----------------------+--------------+------------+----------------+-------------+--------+--------------+-------+
|state|city|HotelName|Address|PhoneNumber       |HotelWebsiteUrl|latitude          |longitude        |HotelRating|Spa|Airport shuttle|Private Lounge|Business Center|Contactless check|Pool|Pet friendly|Free Wifi|Wheel-chair accessible|Snack Bar|Complementary breakfast|Fitness Center|Eco-friendly|24 hr front desk|Valet Parking|Open bar|Child Friendly|smoking|
+-----+----+---------+-------+------------------+---------------+------------------+-----------------+-----------+---+---------------+--------------+---------------+-----------------+----+------------+---------+----------------------+---------+-----------------------+------------

In [74]:
# List of columns to drop
dropping_cols = ['Private Lounge', 'Snack Bar', 'Eco-friendly', 'Child Friendly']

# Drop the specified columns
final_spark_dropped = final_spark.drop(*dropping_cols)

# Show the result
final_spark_dropped.show(truncate=False)

+-----+----------+-------------------------------------------------------------------------------+-----------------------------------------------------------------------+-----------+-------------------------------------------------------------------------------------------------------------------------------+------------------+-------------------+-----------+---+---------------+---------------+-----------------+----+------------+---------+----------------------+-----------------------+--------------+----------------+-------------+--------+-------+
|state|city      |HotelName                                                                      |Address                                                                |PhoneNumber|HotelWebsiteUrl                                                                                                                |latitude          |longitude          |HotelRating|Spa|Airport shuttle|Business Center|Contactless check|Pool|Pet friendly|Free Wifi|Whe

In [76]:
# Coalesce to a single partition and save as a single CSV file
final_spark.coalesce(1).write.csv("hotel_file.csv", header=True, mode="overwrite")


In [77]:
!git init


Reinitialized existing Git repository in /content/.git/


In [79]:
!git remote add origin 'https://github.com/naveen1600/Travel-Advisor.git'

In [81]:
!git add '/content/drive/MyDrive/Colab Notebooks/Preprocessing_hotels.ipynb'

In [82]:
!git status

On branch master

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)
	new file:   drive/MyDrive/Colab Notebooks/Preprocessing_hotels.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.config/
	city_state.json
	drive/MyDrive/Colab Notebooks/FurtherPreprocessing.ipynb
	drive/MyDrive/Colab Notebooks/Untitled0.ipynb
	drive/MyDrive/Colab Notebooks/Untitled1.ipynb
	drive/MyDrive/Colab Notebooks/Untitled2.ipynb
	drive/MyDrive/Colab Notebooks/Untitled3.ipynb
	drive/MyDrive/Colab Notebooks/Vidya's code.ipynb
	drive/MyDrive/Colab Notebooks/finalHotelRecommendation.ipynb
	drive/MyDrive/Colab Notebooks/hotels_EDA.ipynb
	drive/MyDrive/Colab Notebooks/proj2.ipynb
	drive/MyDrive/HotelDatasetFinal.csv
	drive/MyDrive/flowers/
	drive/MyDrive/flowers_model.keras
	drive/MyDrive/hotels.csv
	drive/MyDrive/hotels.zip
	drive/MyDrive/hotels_cleaned.csv
	drive/MyDrive/model/
	drive/MyDrive/output_file.json/
	drive/MyDrive/proj3_test.py

In [84]:
!git branch -M main

In [85]:
!git status

On branch main

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)
	new file:   drive/MyDrive/Colab Notebooks/Preprocessing_hotels.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.config/
	city_state.json
	drive/MyDrive/Colab Notebooks/FurtherPreprocessing.ipynb
	drive/MyDrive/Colab Notebooks/Untitled0.ipynb
	drive/MyDrive/Colab Notebooks/Untitled1.ipynb
	drive/MyDrive/Colab Notebooks/Untitled2.ipynb
	drive/MyDrive/Colab Notebooks/Untitled3.ipynb
	drive/MyDrive/Colab Notebooks/Vidya's code.ipynb
	drive/MyDrive/Colab Notebooks/finalHotelRecommendation.ipynb
	drive/MyDrive/Colab Notebooks/hotels_EDA.ipynb
	drive/MyDrive/Colab Notebooks/proj2.ipynb
	drive/MyDrive/HotelDatasetFinal.csv
	drive/MyDrive/flowers/
	drive/MyDrive/flowers_model.keras
	drive/MyDrive/hotels.csv
	drive/MyDrive/hotels.zip
	drive/MyDrive/hotels_cleaned.csv
	drive/MyDrive/model/
	drive/MyDrive/output_file.json/
	drive/MyDrive/proj3_test.py
	

In [86]:
!git commit -m "Hotel preprocessing1 Aishwarya"


Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@e97fdf480588.(none)')
